<a href="https://colab.research.google.com/github/shreyawalia/ai-powered-file-management/blob/main/train%26val_sets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install iterative-stratification

In [ ]:
import numpy as np
import pandas as pd 
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold, MultilabelStratifiedShuffleSplit
from sklearn.model_selection import ShuffleSplit, train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from pathlib import Path
import os

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
base_dir = "gdrive/My Drive/sdg/dataset/"
labelled_dataset = base_dir + "sdg_tag.csv"
output_dir = "gdrive/My Drive/sdg/dataset/cross_validation/"

Mounted at /content/gdrive


In [ ]:
data_df = pd.read_csv(labelled_dataset)
#mlb = MultiLabelBinarizer(classes = ("goal_1", "goal_2", "goal_3", "goal_4", "goal_5", "goal_6", "goal_7", "goal_8", "goal_9", "goal_10", "goal_11", "goal_12", "goal_13", "goal_14", "goal_15", "goal_16", "goal_17"))

mask = data_df['Tag'].notnull()

data_df.Tag = data_df.loc[mask, 'Tag'].dropna().str.strip('[]').str.split(',')
#data_df.Tag = data_df.Tag.str.split(',').apply(lambda x: [int(i) for i in x])


mskf = MultilabelStratifiedKFold(n_splits=5, random_state=0)
#mlb = MultiLabelBinarizer()
mlb = MultiLabelBinarizer(classes = ("goal_1", "goal_2", "goal_3", "goal_4", "goal_5", "goal_6", "goal_7", "goal_8", "goal_9", "goal_10", "goal_11", "goal_12", "goal_13", "goal_14", "goal_15", "goal_16", "goal_17"))

count = 0

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [ ]:
x = data_df[['description']].values # text
y = mlb.fit_transform(data_df.Tag) # labels

for train_index, test_index in mskf.split(x, y):
    count += 1
    print(f"Fold no. {count}")
    fold_dir = Path(f"{output_dir}fold_{count}/")
    fold_dir.mkdir(exist_ok=True)
    np.save(fold_dir/"train", train_index)
    np.save(fold_dir/"test", test_index)

In [ ]:
data_df.head()

,Unnamed: 0,description,goal_1,goal_2,goal_3,goal_4,goal_5,goal_6,goal_7,goal_8,goal_9,goal_10,goal_11,goal_12,goal_13,goal_14,goal_15,goal_16,goal_17,Tag
0,0,Description\nBackground:\nIn order to achieve ...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,"[goal_1, goal_2, goal_3, goal_4, goal_5, goal_..."
1,1,Description/achievement of initiative\nDesa Wi...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,"[goal_8, goal_15]"
2,2,Description\nThe overall objective of this pro...,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,"[goal_13, goal_14]"
3,3,Description\nThe Global Goals Jam is a two day...,0,0,1,1,0,0,0,0,0,1,1,0,1,1,1,0,1,"[goal_3, goal_4, goal_10, goal_11, goal_13, go..."
4,4,Description/achievement of initiative\nNewport...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,[goal_4]


In [ ]:
data_df.info

<bound method DataFrame.info of       Unnamed: 0  ...                                        Tag
0              0  ...  1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
1              1  ...                                       8,15
2              2  ...                                      13,14
3              3  ...                      3,4,10,11,13,14,15,17
4              4  ...                                          4
...          ...  ...                                        ...
1929        1929  ...                                  4,8,12,14
1930        1930  ...                 4,6,8,11,12,13,14,15,16,17
1931        1931  ...                                   1,5,8,16
1932        1932  ...                                          3
1933        1933  ...                                         16

[1934 rows x 20 columns]>

In [ ]:
data_df['Tag'].to_numeric()